In [0]:
query = """
select * from main_prod.datascience_scratchpad.nyc_test_filtered
"""

In [0]:
df = spark.sql(query)
display(df)

In [0]:
# df.count(), df.select('traj_id').distinct().count()

In [0]:
def data_check(df):
  a = df.count()
  b = df.select('traj_id').distinct().count()
  assert a == b, "Data check failed"


In [0]:
query = """
select userid, date(max(lastupdatedon)) as last_emp_start_date 
from ( SELECT
    userid,
    lastupdatedon,
    employerid,
    LAG(employerid) OVER (PARTITION BY userid ORDER BY lastupdatedon) AS prev_employerid,
    LAG(lastupdatedon) OVER (PARTITION BY userid ORDER BY lastupdatedon) AS prev_lastupdatedon
  FROM main_prod.payroll.usertopayrollhistory
  where payrollstatusid = 11
  )
where prev_employerid != employerid and employerid is not null
group by userid
"""

df = spark.sql(query)
display(df)




In [0]:
df.createOrReplaceTempView("temp_df")

In [0]:
query = """
select df.*, t.last_emp_start_date as last_emp_start_date from main_prod.datascience_scratchpad.nyc_test_filtered as df join temp_df as t on df.userid = t.userid
"""

new_df = spark.sql(query)
display(new_df)

In [0]:
# new_df.count()

In [0]:
# new_df.select('userid').distinct().count()

In [0]:
new_df.createOrReplaceTempView("new_df")

In [0]:
query = """
select distinct a.* from main_prod.earnings_analysis.fact_user_earnings_daily as a join new_df as b on a.userid = b.userid
"""
df = spark.sql(query)
display(df)

In [0]:
# userids = df.select('userid').distinct().collect()
# userids = [x.userid for x in userids]
# print(userids)



In [0]:
df.createOrReplaceTempView('pck_df')

In [0]:
query = """
select userid, paydate, mean(total_pck_amt) as total_pck_amt from pck_df where paydate is not NULL group by userid, paydate order by userid
"""
pck_df = spark.sql(query)
display(pck_df)

In [0]:
# pck_df.select('userid').distinct().count()

In [0]:
pck_df.createOrReplaceTempView('pck_df')

In [0]:
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import udf, col, lit, to_date
from pyspark.sql.types import DateType

def convert_todate(s):
    userid, date_str = s.split("_")
    date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
    return date_obj

spark.udf.register('convert_todate', convert_todate, DateType())
query = """
select *, convert_todate(traj_id) as traj_date from new_df
"""
df = spark.sql(query)
display(df)

In [0]:
df_filtered = df.filter(col("last_emp_start_date")<=col("traj_date"))
display(df_filtered)

In [0]:
print(data_check(df_filtered))

In [0]:
# df_filtered.count()

In [0]:
# df_filtered.select('userid').distinct().count()

In [0]:
df_filtered.createOrReplaceTempView("df_filtered")

In [0]:
query = """
select userid, min(traj_date) as min_date, max(traj_date) as max_date from df_filtered group by userid
"""
min_max_ts_df = spark.sql(query)
display(min_max_ts_df)

In [0]:
df_v2 = df_filtered.join(min_max_ts_df, on='userid')
display(df_v2)
# df_v2.count()
# df_v2.select('userid').distinct().count()

In [0]:
data_check(df_v2)

In [0]:
df_v2.createOrReplaceTempView("df_v2")

In [0]:
# remove userid with less than 50 traj
query = """
select userid, count(*) as traj_count from df_v2 group by userid having traj_count >= 50
"""
frequent_users_df = spark.sql(query)
display(frequent_users_df)
frequent_users_df.select('userid').distinct().count()
frequent_users_df.createOrReplaceTempView("frequent_users_df")

In [0]:
df_v2_filtered = df_v2.join(frequent_users_df, on='userid')
display(df_v2_filtered)
# df_v2_filtered.count()

In [0]:
# userid_pck_df_dict = {}
# userids = df_v2_filtered.select('userid').distinct().toPandas()['userid'].tolist()


In [0]:
# for userid in userids:
#     userid_pck_df_dict[userid] = spark.sql("select * from pck_df where userid = {userid} order by paydate".format(userid=userid))

In [0]:
from datetime import datetime
import bisect
def find_next_bigger(sorted_list, number):
    new_sorted_list = [int(datetime.combine(x, datetime.min.time()).timestamp()) for x in sorted_list]
    number = int(datetime.combine(number, datetime.min.time()).timestamp())
    index = bisect.bisect_right(new_sorted_list, number)
    if index>=len(new_sorted_list):
        return None, None
    if index == 0:
        return None, None
    if new_sorted_list[index-1]==number:
        index-=1
    return sorted_list[index], sorted_list[index-1]

def get_last_paydate(userid):
    pck_data = userid_pck_df_dict[userid].toPandas()
    # print(pck_data)
    if len(pck_data)==0:
        return None, None
    max_date = min_max_ts_df.filter(col("userid")==userid).select("max_date").collect()[0][0]
    dates = pck_data['paydate'].tolist()
    pck = pck_data['total_pck_amt'].tolist()
    # print(dates, max_date)
    # search for date just larger than max_date in dates list.
    pck_date, prev_pck_date = find_next_bigger(dates, max_date)
    if pck_date==None:
        return None, None
    return pck_date, prev_pck_date

    


In [0]:
# pck_date, prev_pck_date = get_last_paydate(1349010)

In [0]:
# pck_date, prev_pck_date

In [0]:
!pip install tqdm
from tqdm import tqdm

In [0]:

# # user_ids_list = df_v2_filtered.select('userid').distinct().toPandas()['userid'].tolist()
# pck_date_list = []
# prev_pck_date_list = []


In [0]:

# for userid in tqdm(userids):
#     pck_date, prev_pck_date = get_last_paydate(userid)
#     pck_date_list.append(pck_date)
#     prev_pck_date_list.append(prev_pck_date)

In [0]:
# user_pck_date_df = spark.createDataFrame(pd.DataFrame({'userid': userids, 'last_pck_date': pck_date_list, 'prev_pck_date': prev_pck_date_list}))
# display(user_pck_date_df)



In [0]:
# user_pck_date_df.write.mode("overwrite").saveAsTable("main_prod.datascience_scratchpad.user_pck_date_v2")
user_pck_date_df = spark.read.table("main_prod.datascience_scratchpad.user_pck_date_v2")

In [0]:
user_pck_filtered = user_pck_date_df.filter(col("prev_pck_date").isNotNull())
display(user_pck_filtered
        )

In [0]:
df_v3 = df_v2_filtered.join(user_pck_filtered, on='userid')
display(df_v3)


In [0]:
def get_train_test_tag(row):
    if row['traj_date']>=row['prev_pck_date']:
        return 'test'
    else:
        return 'train'

In [0]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, struct
udf_get_train_test_tag = udf(get_train_test_tag, StringType())
df_v4 = df_v3.withColumn('train_test_tag', udf_get_train_test_tag(struct('traj_date', 'prev_pck_date')))
display(df_v4)

In [0]:
data_check(df_v4)

In [0]:
df_v4.createOrReplaceTempView("df_v4")

In [0]:
userids_df_v4 = df_v4.select('userid').distinct().toPandas()['userid'].tolist()

In [0]:
from tqdm import tqdm
user_pck_df_dict = {}
for userid in tqdm(userids_df_v4):
    query = """
    select * from pck_df where userid={userid} ORDER BY paydate
    """.format(userid=userid)
    df = spark.sql(query)
    user_pck_df_dict[userid] = df


In [0]:
# user_pck_df_dict[1349010].select('paydate').toPandas()['paydate'].tolist()

In [0]:
# df_v4.count()


In [0]:
# df_v4.show()

In [0]:
df_v4.write.mode('overwrite').saveAsTable("main_prod.datascience_scratchpad.df_v4")

In [0]:
pck_df.write.mode('overwrite').saveAsTable('main_prod.datascience_scratchpad.pck_df')

In [0]:
display(df_v4)

In [0]:
df_v4.count()

In [0]:
df_v4.select('traj_id').distinct().count()


In [0]:
df_v4_pd = df_v4.toPandas()
len(df_v4_pd['traj_id'].unique())

In [0]:
user_ids = df_v4_pd.userid.unique()
print(len(user_ids))

In [0]:
filtered_pck_df = pck_df.filter(pck_df['userid'].isin(user_ids))

In [0]:
filtered_pck_df.select('userid').distinct().count()


In [0]:
filtered_pck_df_pd = filtered_pck_df.toPandas()


In [0]:
import bisect
def get_paycheck_amount(userid, traj_date):    
    user_df = filtered_pck_df_pd[filtered_pck_df_pd['userid']==userid].sort_values(by='paydate')
    paydate_list = user_df['paydate'].tolist()
    amount_list = user_df['total_pck_amt'].tolist()
    nearest_paydate_idx = bisect.bisect_right(paydate_list, traj_date)
    if nearest_paydate_idx == 0:
        return amount_list[nearest_paydate_idx]
    elif paydate_list[nearest_paydate_idx - 1] == traj_date:
        return amount_list[nearest_paydate_idx - 1]
    elif nearest_paydate_idx == len(paydate_list):
        return None
    else:
        return amount_list[nearest_paydate_idx]

In [0]:
# apply function in each pandas df row
tqdm.pandas()
def calculate_paycheck_amount(row):
    return get_paycheck_amount(row['userid'], row['traj_date'])

# Apply the function to each row and create a new column
df_v5 = df_v4_pd.copy()
df_v5['paycheck_amount'] = df_v5.progress_apply(calculate_paycheck_amount, axis=1)

# Save the DataFrame to a parquet file




In [0]:
df_v5.to_parquet('/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/nyc/inference_data_v3/df_v5.parquet')

In [0]:
# from glob import glob
# parquet_files = glob('/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/nyc/inference_data/*.parquet')
# df_v5 = spark.read.parquet(*parquet_files)
# df_v5.count()

In [0]:
df_v5_spark = spark.createDataFrame(df_v5)

In [0]:
display(df_v5)

In [0]:
df_v5['userid'].drop_duplicates().count()

In [0]:
filtered_pck_df_pd[filtered_pck_df_pd['userid']==1485347].sort_values(by='paydate')


In [0]:
df_v5.write.mode("overwrite").saveAsTable("main_prod.datascience_scratchpad.traj_inf_df")

In [0]:
df_v5 = spark.read.table("main_prod.datascience_scratchpad.traj_inf_df")

df_v5.count()

In [0]:
len(df_v5['traj_id'].unique())


In [0]:
display(df_v6)

In [0]:
query = """
select userid, mean(paycheck_amount) as total_pck from main_prod.datascience_scratchpad.traj_inf_df where train_test_tag='test' group by userid
"""
df_v7 = df_v5[df_v5['train_test_tag']=='test'].groupby('userid').agg({'paycheck_amount':'mean'}).reset_index()



In [0]:
display(df_v7)

In [0]:
df_v7_fil = df_v7[(df_v7['paycheck_amount']==0) | (df_v7['paycheck_amount']>400)].reset_index(drop=True)
display(df_v7_fil)


In [0]:
from pyspark.sql.types import IntegerType
def get_label(total_pck):
    if total_pck == 0:
        return 0
    else:
        return 1
    
df_v7_fil['label']= df_v7_fil['paycheck_amount'].apply(get_label)


In [0]:
display(df_v7_fil)

In [0]:
df_v8 = df_v5[df_v5['userid'].isin(df_v7_fil['userid'].unique())].reset_index(drop=True)

In [0]:
len(df_v5['userid'].unique())

In [0]:
len(df_v8['userid'].unique())

In [0]:
def get_userid_label(userid):
    label = df_v7_fil[df_v7_fil['userid']==userid]['label'].values[0]
    return label

df_v9 = df_v8.copy()
df_v9['label'] = df_v9['userid'].progress_apply(get_userid_label)

In [0]:
df_v9 = df_v9[['userid', 'traj_id','train_test_tag', 'timestamps_filtered', 'wgs_seq_filtered', 'merc_seq_filtered', 'trajlen','paycheck_amount', 'label']]

In [0]:
len(df_v9['traj_id'].unique())

In [0]:
df_v9

In [0]:
df_v9.to_parquet("/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/nyc/inference_data_v3/traj_test_df_v3.parquet")

In [0]:
%sql
select * from main_prod.earnings_analysis.fact_user_earnings_daily where userid = "34299"

In [0]:
%sql
select * from main_prod.paycheckprediction.receivedpaycheck where userid  = "34299"

In [0]:
%sql
select * from main_prod.analysis.derived_userstate_bankconnection where userid = "131568"